![Los Angeles skyline](../../imgs/la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [9]:
import pandas as pd

## So as to not have to upload the csv, download it and read into the dataframe instead
url = "https://data.lacity.org/resource/63jg-8b9z.csv?$query=SELECT%20dr_no%2C%20date_rptd%2C%20date_occ%2C%20time_occ%2C%20area%2C%20area_name%2C%20rpt_dist_no%2C%20part_1_2%2C%20crm_cd%2C%20crm_cd_desc%2C%20mocodes%2C%20vict_age%2C%20vict_sex%2C%20vict_descent%2C%20premis_cd%2C%20premis_desc%2C%20weapon_used_cd%2C%20weapon_desc%2C%20status%2C%20status_desc%2C%20crm_cd_1%2C%20crm_cd_2%2C%20crm_cd_3%2C%20crm_cd_4%2C%20location%2C%20cross_street%2C%20lat%2C%20lon"

crimes = pd.read_csv(url, parse_dates=["date_rptd", "date_occ"], dtype={"time_occ": str})
crimes = crimes.rename(columns={"dr_no": "DR_NO", 
                                "date_rptd": "Date Rptd", 
                                "date_occ": "DATE OCC", 
                                "time_occ": "TIME OCC", 
                                "area_name": "AREA NAME", 
                                "crm_cd_desc": "Crm Cd Desc", 
                                "vict_age": "Vict Age", 
                                "vict_sex": "Vict Sex", 
                                "vict_descent": "Vict Descent", 
                                "weapon_desc": "Weapon Desc", 
                                "status_desc": "Status Desc", 
                                "location": "LOCATION"})

cols = ["DR_NO", "Date Rptd", "DATE OCC", "TIME OCC", 
        "AREA NAME", "Crm Cd Desc", "Vict Age", "Vict Sex", 
        "Vict Descent", "Weapon Desc", "Status Desc", 
        "LOCATION"]
crimes = crimes[cols]

In [10]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,1307355,2010-02-20,2010-02-20,1350,Newton,VIOLATION OF COURT ORDER,48,M,H,NaN,Adult Arrest,300 E GAGE AV
1,11401303,2010-09-13,2010-09-12,0045,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,NaN,Invest Cont,SEPULVEDA BL
2,70309629,2010-08-09,2010-08-09,1515,Newton,OTHER MISCELLANEOUS CRIME,0,M,H,NaN,Invest Cont,1300 E 21ST ST
3,90631215,2010-01-05,2010-01-05,0150,Hollywood,VIOLATION OF COURT ORDER,47,F,W,HAND GUN,Invest Cont,CAHUENGA BL
4,100100501,2010-01-03,2010-01-02,2100,Central,"RAPE, ATTEMPTED",47,F,H,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,8TH ST


In [11]:

# Which hour has the highest frequency of crimes? 
crimes["hour"] = (np.floor(crimes['TIME OCC'].astype(int) / 100)).astype(int)
peak_crime_hour = crimes["hour"].value_counts().index[0].item()
print(" - The peak crime hour is :", peak_crime_hour)


# Which area has the largest frequency of night crimes (crimes committed between 10pm and 3:59am)?
night_crimes = crimes[(crimes["hour"] >= 2200) | (crimes["hour"] <= 400)]
peak_night_crime_location = night_crimes["AREA NAME"].value_counts().index[0]
print(" - The peak night (2200 to 0359) crime area is :", peak_night_crime_location)



## The better way of solving the problem
age_bins = [0, 17, 25, 34, 44, 54, 64, np.inf]
age_labels = ["<18", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]
crimes["Age Bracket2"] = pd.cut(crimes["Vict Age"],
                               bins=age_bins,
                               labels=age_labels)
victim_ages = crimes["Age Bracket2"].value_counts()
print(victim_ages)


# Identify the number of crimes committed against victims by age group (<18, 18-25, 26-34, 35-44, 45-54, 55-64, 65+).
## A less effective way of solving this problem
crimes.loc[(crimes["Vict Age"] >= 1) & (crimes["Vict Age"] < 18), "Age Bracket"] = "<18"
crimes.loc[(crimes["Vict Age"] >= 18) & (crimes["Vict Age"] <= 25), "Age Bracket"] = "18-25"
crimes.loc[(crimes["Vict Age"] >= 26) & (crimes["Vict Age"] <= 34), "Age Bracket"] = "26-34"
crimes.loc[(crimes["Vict Age"] >= 35) & (crimes["Vict Age"] <= 44), "Age Bracket"] = "35-44"
crimes.loc[(crimes["Vict Age"] >= 45) & (crimes["Vict Age"] <= 54), "Age Bracket"] = "45-54"
crimes.loc[(crimes["Vict Age"] >= 55) & (crimes["Vict Age"] <= 64), "Age Bracket"] = "55-64"
crimes.loc[crimes["Vict Age"] >= 65, "Age Bracket"] = "65+"
victim_ages = crimes["Age Bracket"].value_counts()
print(" - Age range of victims is :")
print(victim_ages)

 - The peak crime hour is : 18
 - The peak night (2200 to 0359) crime area is : Central
Age Bracket2
26-34    229
18-25    199
35-44    181
45-54    160
55-64     82
<18       37
65+       15
Name: count, dtype: int64
 - Age range of victims is :
Age Bracket
26-34    229
18-25    199
35-44    181
45-54    160
55-64     82
<18       37
65+       15
Name: count, dtype: int64


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION,hour,Age Bracket2,Age Bracket
0,1307355,2010-02-20,2010-02-20,1350,Newton,VIOLATION OF COURT ORDER,48,M,H,NaN,Adult Arrest,300 E GAGE AV,13,45-54,45-54
1,11401303,2010-09-13,2010-09-12,0045,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,NaN,Invest Cont,SEPULVEDA BL,0,NaN,NaN
2,70309629,2010-08-09,2010-08-09,1515,Newton,OTHER MISCELLANEOUS CRIME,0,M,H,NaN,Invest Cont,1300 E 21ST ST,15,NaN,NaN
3,90631215,2010-01-05,2010-01-05,0150,Hollywood,VIOLATION OF COURT ORDER,47,F,W,HAND GUN,Invest Cont,CAHUENGA BL,1,45-54,45-54
4,100100501,2010-01-03,2010-01-02,2100,Central,"RAPE, ATTEMPTED",47,F,H,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,8TH ST,21,45-54,45-54
5,100100506,2010-01-05,2010-01-04,1650,Central,SHOPLIFTING - PETTY THEFT ($950 & UNDER),23,M,B,NaN,Adult Arrest,700 W 7TH ST,16,18-25,18-25
6,100100508,2010-01-08,2010-01-07,2005,Central,BURGLARY FROM VEHICLE,46,M,H,NaN,Invest Cont,PICO BL,20,45-54,45-54
7,100100509,2010-01-09,2010-01-08,2100,Central,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",51,M,B,UNKNOWN WEAPON/OTHER WEAPON,Adult Arrest,500 CROCKER ST,21,45-54,45-54
8,100100510,2010-01-09,2010-01-09,0230,Central,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",30,M,H,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,800 W OLYMPIC BL,2,26-34,26-34
9,100100511,2010-01-09,2010-01-06,2100,Central,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",55,M,W,NaN,Invest Cont,200 S OLIVE ST,21,55-64,55-64
